# BI-geomorph-extraction

Extract barrier island metrics along transects for Bayesian Network Deep Dive

Requires: python 3, ArcPy

Author: Emily Sturdivant

email: esturdivant@usgs.gov; bgutierrez@usgs.gov

Notes:
- Run in ArcGIS Pro python 3 environment (access as: \ArcGIS\Pro\bin\Python\Scripts\proenv);
- Spatial reference used is NAD 83 UTM 19N: arcpy.SpatialReference(26918)
    
### Import modules

In [2]:
import os
import sys
import time
import shutil
import pandas as pd
import numpy as np
import arcpy
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import CoastalVarExtractor.functions_warcpy as fwa
import CoastalVarExtractor.functions as fun

If you refuse to install:

```python
mod_path = "C:\Users\esturdivant\Code\BI-geomorph-extraction" # replace with path to module
sys.path.append(mod_path)
import CoastalVarExtractor.functions_warcpy as fwa
import CoastalVarExtractor.functions as fun
```

### Initialize variables

Based on the project directory, and the site and year you have input, setvars.py will set a bunch of variables as the names of folders, files, and fields. 1) set-up the project folder and paths: 

In [3]:
from CoastalVarExtractor.setvars import *

site: Fisherman
year: 2014
setvars.py initialized variables.


## Transect-averaged values

In [3]:
#%% Create trans_df
trans_df = fwa.FCtoDF(extendedTrans, id_fld=tID_fld, extra_fields=extra_fields)
trans_df.to_pickle(os.path.join(scratch_dir, 'trans_df.pkl'))
print("Header of transects dataframe (rows 1-5 out of {}: ".format(len(trans_df)))
trans_df.head()

Converting feature class to array...
Converting array to dataframe...
Header of transects dataframe (rows 1-5 out of 194: 


,Azimuth,LCI90,LR2,LRR,LSE,TransOrder,TransectId,sort_ID
sort_ID,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [4]:
print("Header of transects dataframe (rows 1-5 out of {}): ".format(len(trans_df)))
trans_df.head()
# Why is the length 175, but the sort_ID values go up to 180?

Header of transects dataframe (rows 1-5 out of 194): 


,Azimuth,LCI90,LR2,LRR,LSE,TransOrder,TransectId,sort_ID
sort_ID,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


### Add XY and Z/slope from DH, DL, SL points within 10m of transects

In [5]:
#%% Add XY and Z/slope from DH, DL, SL points within 10m of transects
sl2trans_df = fwa.add_shorelinePts2Trans(extendedTrans, ShorelinePts, shoreline, tID_fld, proximity=pt2trans_disttolerance)
sl2trans_df.to_pickle(os.path.join(scratch_dir, 'sl2trans.pkl'))
fwa.DFtoFC(sl2trans_df, os.path.join(arcpy.env.scratchGDB, 'pts2trans_SL'), spatial_ref=utmSR, id_fld=tID_fld, xy=["SL_x", "SL_y"], keep_fields=['Bslope'])

dh2trans_df = fwa.find_ClosestPt2Trans_snap(extendedTrans, dhPts, trans_df, 'DH', tID_fld, proximity=pt2trans_disttolerance)
dh2trans_df.to_pickle(os.path.join(scratch_dir, 'dh2trans.pkl'))
fwa.DFtoFC(dh2trans_df, os.path.join(arcpy.env.scratchGDB, 'ptSnap2trans_DH'), spatial_ref=utmSR, id_fld=tID_fld, xy=["DH_snapX", "DH_snapY"], keep_fields=['DH_z'])

dl2trans_df = fwa.find_ClosestPt2Trans_snap(extendedTrans, dlPts, trans_df, 'DL', tID_fld, proximity=pt2trans_disttolerance)
dl2trans_df.to_pickle(os.path.join(scratch_dir, 'dl2trans.pkl'))
fwa.DFtoFC(dl2trans_df, os.path.join(arcpy.env.scratchGDB, 'ptSnap2trans_DL'), spatial_ref=utmSR, id_fld=tID_fld, xy=["DL_snapX", "DL_snapY"], keep_fields=['DL_z'])

arm2trans_df = fwa.ArmorLineToTrans_PD(extendedTrans, armorLines, sl2trans_df, tID_fld, proj_code, elevGrid_5m)
arm2trans_df.to_pickle(os.path.join(scratch_dir, 'arm2trans.pkl'))

#%% Add all the positions to the trans_df
trans_df = fun.join_columns_id_check(trans_df, sl2trans_df, tID_fld)
trans_df = fun.join_columns_id_check(trans_df, dh2trans_df, tID_fld)
trans_df = fun.join_columns_id_check(trans_df, dl2trans_df, tID_fld)
trans_df = fun.join_columns_id_check(trans_df, arm2trans_df, tID_fld)
trans_df.to_pickle(os.path.join(scratch_dir, 'trans_df_beachmetrics.pkl'))

trans_df.head()


Joining shoreline points to transects...
...duration at transect 100: 0:0:21.8 seconds
Duration: 0:0:41.5 seconds

Joining DH points to transects:
Getting name of Z field...
Looking for field _z
Looping through transects to find nearest point within 25 meters...
Duration at transect 100: 0:0:14.5 seconds
Duration: 0:0:26.4 seconds

Joining DL points to transects:
Getting name of Z field...
Looking for field _z
Looping through transects to find nearest point within 25 meters...
Duration at transect 100: 0:0:13.4 seconds
Duration: 0:0:24.3 seconds

Armoring file either missing or empty so we will proceed without armoring data. If shorefront tampering is present at this site, cancel the operations to digitize.


,Azimuth,LCI90,LR2,LRR,LSE,TransOrder,TransectId,SL_x,SL_y,Bslope,...,DH_snapX,DH_snapY,DL_x,DL_y,DL_z,DL_snapX,DL_snapY,Arm_x,Arm_y,Arm_z
sort_ID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,413851.177269,4.107531e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,413809.698456,4.107514e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,413778.475432,4.107478e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,413746.021688,4.107456e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,413702.743869,4.107442e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Add all the positions to the trans_df

In [7]:
# trans_df = pd.read_pickle(os.path.join(scratch_dir, 'trans_df_beachmetrics.pkl'))


In [9]:
#%% Calculate distances from shore to dunes, etc.
trans_df, dl2trans, dh2trans, arm2trans = fwa.calc_BeachWidth_fill(extendedTrans, trans_df, maxDH, tID_fld, MHW, fill)
trans_df.loc[90:100]


,Azimuth,LCI90,LR2,LRR,LSE,TransOrder,TransectId,SL_x,SL_y,Bslope,...,Arm_z,DH_zmhw,DL_zmhw,Arm_zmhw,DistDL,DistDH,DistArm,uBW,uBH,ub_feat
sort_ID,,,,,,,,,,,,,,,,,,,,,
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414630.728924,4.104552e+06,-0.068306,...,NaN,1.293877,NaN,NaN,NaN,63.856862,NaN,63.856862,1.293877,DH
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414679.813678,4.104541e+06,NaN,...,NaN,1.263099,NaN,NaN,NaN,75.779069,NaN,75.779069,1.263099,DH
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414728.642394,4.104532e+06,NaN,...,NaN,1.465732,0.746082,NaN,70.673444,81.774756,NaN,70.673444,0.746082,DL
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414776.212892,4.104520e+06,-0.077578,...,NaN,1.548900,0.745265,NaN,75.809086,86.913286,NaN,75.809086,0.745265,DL
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414820.696513,4.104512e+06,-0.081250,...,NaN,1.493574,0.753078,NaN,81.953619,93.194459,NaN,81.953619,0.753078,DL
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414869.829538,4.104505e+06,-0.075188,...,NaN,1.541450,0.805521,NaN,89.391596,105.597779,NaN,89.391596,0.805521,DL
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414918.577682,4.104502e+06,-0.071777,...,NaN,2.795067,1.158149,NaN,141.149873,147.368012,NaN,141.149873,1.158149,DL
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414962.190849,4.104498e+06,-0.073646,...,NaN,2.023152,0.970792,NaN,138.418872,147.095728,NaN,138.418872,0.970792,DL
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,415016.319309,4.104488e+06,-0.040494,...,NaN,2.274932,1.471800,NaN,145.581507,154.236022,NaN,145.581507,1.471800,DL


### Dist2Inlet

In [10]:
# Dist2Inlet: Calc dist from inlets 
dist_df = fwa.measure_Dist2Inlet(shoreline, extendedTrans, inletLines, tID_fld)
dist_df.to_pickle(os.path.join(scratch_dir, 'dist2inlet_df.pkl'))
trans_df = fun.join_columns_id_check(trans_df, dist_df, tID_fld, fill=fill)

Duration: 0:0:1.9 seconds


In [23]:
# QC: difference in dist2inlet between consecutive transects.
trans_df.loc[14:17, 'Dist2Inlet']

sort_ID
14     88.106911
15    298.846926
16    388.604316
17    465.575256
Name: Dist2Inlet, dtype: float64

### Clip transects, get barrier widths *SPATIAL*

In [12]:
# Clip transects, get barrier widths *SPATIAL*
widths_df = fwa.calc_IslandWidths(extendedTrans, barrierBoundary, tID_fld=tID_fld)
trans_df = fun.join_columns_id_check(trans_df, widths_df, tID_fld, fill=fill)
trans_df.to_pickle(os.path.join(scratch_dir, extTrans_null+'_prePts.pkl'))
# trans_df = pd.read_pickle(os.path.join(scratch_dir, extTrans_null+'_prePts.pkl'))

Clipping the transects to the barrier island boundaries...
Getting the width along each transect of the oceanside land (WidthPart)...
Converting feature class to array...
Converting array to dataframe...
Getting the width along each transect of the entire barrier (WidthFull)...
Converting feature class vertices to array with X and Y...
Converting array to dataframe...
Getting the width along each transect of above water portion of the barrier (WidthLand)...


## 5m Points

In [14]:
# trans_df = pd.read_pickle(os.path.join(scratch_dir, extTrans_null+'_prePts.pkl'))
transPts_presort


'\\\\Mac\\stor\\Projects\\TransectExtraction\\Fisherman2014\\scratch.gdb\\tran5mPts_unsorted'

In [13]:
# if os.path.exists(os.path.join(scratch_dir, transPts_null+'.pkl')):
#     pts_df = pd.read_pickle(os.path.join(scratch_dir,transPts_null+'.pkl'))
#     trans_df = pd.read_pickle(os.path.join(scratch_dir, extTrans_null+'_prePts.pkl'))
pts_df, transPts_presort = fwa.TransectsToPointsDF(extTrans_tidy, barrierBoundary, fc_out=transPts_presort) # 4 minutes for FireIsland

Getting points every 5m along each transect and saving in dataframe...
Converting new dataframe to feature class...
Duration: 0:1:25.1 seconds


In [15]:
# if not 'ptZ' in pts_df.columns:
    # Extract elevation and slope at points
if not arcpy.Exists(elevGrid_5m):
    fwa.ProcessDEM(elevGrid, elevGrid_5m, utmSR)
if not arcpy.Exists(slopeGrid):
    arcpy.Slope_3d(elevGrid_5m, slopeGrid, 'PERCENT_RISE')
arcpy.sa.ExtractMultiValuesToPoints(transPts_presort, 
                                    [[elevGrid_5m, 'ptZ'], [slopeGrid, 'ptSlp']]) # 9 min for ParkerRiver

<geoprocessing server result object at 0xba33e90>

In [16]:
pts_df = fwa.FCtoDF(transPts_presort, xy=True, dffields=[tID_fld,'ptZ', 'ptSlp'])
pts_df.to_pickle(os.path.join(scratch_dir, 'pts_df_elev_slope.pkl'))

pts_df.head()

Converting feature class to array with X and Y...
Converting array to dataframe...


,SHAPE@X,SHAPE@Y,ptSlp,ptZ,sort_ID
0,413851.177269,4.107531e+06,5.669233,-0.162717,1.0
1,413853.654371,4.107527e+06,5.935968,-0.043215,1.0
2,413856.131473,4.107523e+06,22.361212,0.712121,1.0
3,413858.608576,4.107518e+06,21.225933,1.064813,1.0
4,413861.085678,4.107514e+06,26.451097,2.700293,1.0


In [17]:
pts_df = pd.read_pickle(os.path.join(scratch_dir, 'pts_df_elev_slope.pkl'))
pts_df.head()

,SHAPE@X,SHAPE@Y,ptSlp,ptZ,sort_ID
0,413851.177269,4.107531e+06,5.669233,-0.162717,1.0
1,413853.654371,4.107527e+06,5.935968,-0.043215,1.0
2,413856.131473,4.107523e+06,22.361212,0.712121,1.0
3,413858.608576,4.107518e+06,21.225933,1.064813,1.0
4,413861.085678,4.107514e+06,26.451097,2.700293,1.0


### Calculate DistSeg, Dist_MHWbay, DistSegDH, DistSegDL, DistSegArm, sort points

In [18]:
#%%
# Calculate DistSeg, Dist_MHWbay, DistSegDH, DistSegDL, DistSegArm, sort points
pts_df = fun.join_columns(pts_df, trans_df, tID_fld)
pts_df = fun.prep_points(pts_df, tID_fld, pID_fld, MHW, fill)
# Aggregate ptZmhw to max and mean and join to transPts and extendedTransects
pts_df, zmhw = fun.aggregate_z(pts_df, MHW, tID_fld, 'ptZ', fill)
trans_df = fun.join_columns(trans_df, zmhw) # join new fields to transects
pts_df = fun.join_columns(pts_df, trans_df, tID_fld) # Join transect values to pts

# Housecleaning
trans_df.drop(extra_fields, axis=1, inplace=True, errors='ignore') # Drop extra fields
pts_df.drop(extra_fields, axis=1, inplace=True, errors='ignore') # Drop extra fields

#%% Save dataframes to open elsewhere or later
trans_df.to_pickle(os.path.join(scratch_dir, extTrans_null+'.pkl'))
pts_df.to_pickle(os.path.join(scratch_dir, transPts_null+'.pkl'))

In [19]:
pts_df = pd.read_pickle(os.path.join(scratch_dir, transPts_null+'.pkl'))
trans_df = pd.read_pickle(os.path.join(scratch_dir, extTrans_null+'.pkl'))
pts_df.head()

,SplitSort,seg_x,seg_y,ptSlp,ptZ,sort_ID,Dist_Seg,DistSegArm,DistSegDH,DistSegDL,...,DistArm,uBW,uBH,ub_feat,Dist2Inlet,WidthFull,WidthLand,WidthPart,mean_Zmhw,max_Zmhw
0,0,413851.177269,4.107531e+06,5.669233,-0.162717,1,5.963374e-09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,79.016858,126.319423,126.319423,126.319423,2.523506,3.498424
1,1,413853.654371,4.107527e+06,5.935968,-0.043215,1,5.000000e+00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,79.016858,126.319423,126.319423,126.319423,2.523506,3.498424
2,2,413856.131473,4.107523e+06,22.361212,0.712121,1,1.000000e+01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,79.016858,126.319423,126.319423,126.319423,2.523506,3.498424
3,3,413858.608576,4.107518e+06,21.225933,1.064813,1,1.500000e+01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,79.016858,126.319423,126.319423,126.319423,2.523506,3.498424
4,4,413861.085678,4.107514e+06,26.451097,2.700293,1,2.000000e+01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,79.016858,126.319423,126.319423,126.319423,2.523506,3.498424


## Quality checking


In [20]:
# 140 is a good one.
trans_in = int(input('Transect ID ({:d}-{:d}): '.format(int(pts_df[tID_fld].head(1)), int(pts_df[tID_fld].tail(1)))))
pts_set = pts_df[pts_df[tID_fld] == trans_in]
# plot
fig = plt.figure(figsize=(13,10)) # Set the size of your figure, customize for more subplots
ax = fig.add_subplot(211)
fun.plot_island_profile(ax, pts_set, MHW, MTL)
ax = fig.add_subplot(212)
fun.plot_beach_profile(ax, pts_set, MHW, MTL)
# display
plt.show()
plt.close('all')

Transect ID (1-194): 50


ValueError: posx and posy should be finite values

## Outputs

In [21]:
#%% Join calculated transect values to the transect FC.
trans_fc = fwa.JoinDFtoFC(trans_df, extendedTrans, tID_fld, out_fc=extTrans_fill)
# DeleteExtraFields(trans_fc, trans_flds)
fwa.CopyFCandReplaceValues(trans_fc, fill, None, out_fc=extTrans_null, out_dir=home)
# Save final SHP with fill values
arcpy.FeatureClassToFeatureClass_conversion(trans_fc, scratch_dir, extTrans_shp+'.shp')

OUTPUT: Fisherman2014_extTrans_null


<Result '\\\\Mac\\stor\\Projects\\TransectExtraction\\Fisherman2014\\scratch\\Fisherman2014_extTrans_shp.shp'>

### Save final pts with fill values as CSV

In [22]:
#%% Save final pts with fill values as CSV
if not pID_fld in pts_df.columns:
    pts_df.reset_index(drop=False, inplace=True)
csv_fname = os.path.join(scratch_dir, transPts_fill +'.csv')
pts_df.to_csv(os.path.join(scratch_dir, transPts_fill +'.csv'), na_rep=fill, index=False)
print("OUTPUT: {}".format(csv_fname))

OUTPUT: \\Mac\stor\Projects\TransectExtraction\Fisherman2014\scratch\Fisherman2014_transPts_fill.csv


### Create Beach Width raster by joining DF to ID raster

In [26]:
#%% Create Beach Width raster by joining DF to ID raster
if not arcpy.Exists(rst_transIDpath):
    outEucAll = arcpy.sa.EucAllocation(extTrans_tidy, maximum_distance=50, cell_size=cell_size, source_field=tID_fld)
    outEucAll.save(os.path.basename(rst_transIDpath))
out_rst = fwa.JoinDFtoRaster(trans_df, rst_transID, bw_rst, fill, tID_fld, 'uBW')

OUTPUT: fish14_ubw. Field "Value" is ID and "uBW" is beachwidth.


### Convert pts_df to FC, both pts and trans (pts_fc, trans_fc)

In [23]:
#%% Convert pts_df to FC, both pts and trans (pts_fc, trans_fc)
pts_fc = fwa.DFtoFC_large(pts_df, out_fc=os.path.join(arcpy.env.workspace, transPts_fill), spatial_ref=utmSR, df_id=pID_fld, xy=["seg_x", "seg_y"])
# DeleteExtraFields(pts_fc, pt_flds+trans_flds)
# Save final FCs with null values, final SHP and XLS with fill values
fwa.CopyFCandReplaceValues(pts_fc, fill, None, out_fc=transPts_null, out_dir=home)
arcpy.FeatureClassToFeatureClass_conversion(pts_fc, scratch_dir, transPts_shp+'.shp')
try:
    xls_fname = os.path.join(scratch_dir, transPts_fill +'.xlsx')
    pts_df.to_excel(xls_fname, na_rep=fill, index=False)
    print("OUTPUT: {}".format(xls_fname))
except:
    print("No Excel file created. You'll have to do it yourself from the CSV.")

Converting points DF to FC...
OUTPUT: \\Mac\stor\Projects\TransectExtraction\Fisherman2014\Fisherman2014.gdb\Fisherman2014_transPts_fill
Duration: 0:2:37.6 seconds
OUTPUT: Fisherman2014_transPts_null
No Excel file created. You'll have to do it yourself from the CSV.
